In [ ]:
from src.graph_utils import *
from src.pge import *
import time
import os

In [ ]:
heights = [2,4,8]

scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP", "Genetic"]

initial_params_seed = 123

optimization_iteration_count = 0

base_path = 'results/'

seeds = [111, 222,333,444,555]

In [ ]:
for seed in seeds:
  report_filename = base_path + 'ParameterEncoding_' +  str(seed) + '.txt'
  for height in heights:
    width = height
    print(f'height: {height}, width: {width}, n: {height*width}')
    np.random.seed(seed=seed)
    G, image = generate_problem_instance(height, width)
    print("Image Generated: ",image)
    # plt.imshow(image, cmap=plt.cm.gray)
    # plt.show()
  
    for optimizer_method in scipy_optimizer_methods:
      print(f"Executing QC with {optimizer_method} optimizer for {height}*{height} image.")
      try:
        start_time = time.time()
        success_flag, new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = new_nisq_algo_solver(G, 
                                                                                                               optimizer_method = optimizer_method, 
                                                                                                               initial_params_seed=initial_params_seed)
        new_nisq_tte = (time.time() - start_time)
        print(new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value)
      except:
        print(f"Execution Failed for {optimizer_method} optimizer for {height}*{height} image.")
        continue
      print("New NISQ done for",optimizer_method,"optimizer with a success status :", success_flag)
      print(f"Appending the results of {height}*{height} image using QC with and {optimizer_method} optimizer.")
      row = []
      row.append(int(G.number_of_nodes()))
      row.append(int(height))
      row.append(int(width))
      row.append(success_flag)
      row.append(''.join(map(str, map(int, (new_nisq_solution)))))
      row.append(np.round(new_nisq_tte,6))
      row.append(np.round(new_nisq_cut_value,4))
      row.append(np.round(new_nisq_expectation_value,4))
      row.append(optimization_iteration_count)
      row.append(optimizer_method)
      report_file_obj = open(os.path.join(report_filename),'a+')
      report_file_obj.write('__'.join(map(str,row))+'\n')
      report_file_obj.close()
    print('\n')